# Face Sonification Prototype

ISSUES:
* if you launch a synth (s_new), it's going to take some time until it is up and running: instantiation of a synth is asynchronous
    * we may lose some messages before the synth is ready to play
    * we should:
        * start the synths in advance (maybe paused)
        * pause synths when they are not useful instead of stopping them
* often, we have a video recorded with a frame rate and sensor data that has been recorded with another frame rate
    * PyQtGraph
    * schedule visual events in the time queue
* augmentation of the video with custom plotting, e.g. yourself plotting markers instead of having them plotted by OpenFace
    * discuss if and how

* The timing problem that OpenFace has seems to be related with OpenCV. The tracked video sometimes is sligtly shorter than the original one and sometimes it is longer. It seems that OpenCV has some problems working with some of the codecs.
    * ffmpeg -i video.file -r 30 -vcodec ffv1 -acodec pcm_s16le output_name
        * in my case, this does not work
    * ffmpeg -i video.file -r 30 -vcodec ffv1 output_name
    * split video frames into images if the previous solution fails.
        * ffmpeg

* We can consider the idea of focusing on the AUs that OpenFace detects with good precision and maybe cut off some of the others. We can find this information in the OpenFace paper.
* The smile seems to be in many context the most important source of information. We can consider the idea of designing sonifications to put this element in the foreground.
* The event-based sonification for how I implemented can be confusing for the listener: events are triggered only when the intensity of the AU goes over some thresholds (i.e. 1, 2, 3, 4). This way, the events are able to provide some information on the dinamic behaviour of the sound, but it also means that if an expression stays in the same range for a long time, we don't really have a continuous feedback, as the sound fades out quickly. Another problem is that events are also triggered when the intensity pass from a high range to a lower range; even if this provides some feedback on the dynamic behaviour, it is counterintuitive, because the user would expect the event to be associated to the activation of an expression.
* We can try to use envelope parameters to handle the fall down
* We can try a more hybrid approach. Some AUs will be sonified using an event-based approach that will omit information relative to the intensity, so that one event for expression will be triggered, e.g. one blink is sonified as one drop: we ignore the information on the intensity of the blink; we can do this for many other expression that are not our primary focus. The most important expressions (e.g. the smile), will be sonified somehow continuously, so that the listener would be able to perceive their stationary behaviour as well.

* We could think of a musical sonification when AUs are associated to tracks that would play well together. When an AU is present, the relative track is played. AUs that are incompatible can be mapped to tracks that don's sound well together.
    * Rithmical structured tracks
    * Ambient
* Sonify the movements of the head and of the eyes. We can sonify the degree of variation to the standard direction, so that when e.g. the head is in a standard position, no sound is played.

In [ ]:
import numpy as np
import pandas as pd

import subprocess
import time
import os

## OpenFace workaround
It works, but...
* we have a very long conversion phase
* the feature extraction is much longer

In [ ]:
!mkdir full
!ffmpeg -i full.avi -f image2 full/video-frame%05d.jpg

In [ ]:
!docker exec -it openface build/bin/FeatureExtraction -out_dir files/processed -fdir files/full -aus -gaze -pose -tracked

In [ ]:
# 0 is the maximum quality
!ffmpeg -y -i full.avi -vcodec mpeg4 -q:v 0 -max_muxing_queue_size 2048 full-rigth-codecs.avi

In [ ]:
!ffmpeg -y -i full.avi -vcodec libx264 -preset slow -crf 22 -max_muxing_queue_size 2048 -an full-rigth-codecs.avi

In [ ]:
!ffmpeg -y -i full.avi -vcodec libx264 -preset slow -crf 0 -max_muxing_queue_size 2048 -an full-rigth-codecs.avi

In [ ]:
!ffmpeg -y -i full.avi -vcodec libx264 -preset veryslow -crf 0 -max_muxing_queue_size 2048 -an full-rigth-codecs.avi

In [ ]:
!ffmpeg -y -i full.avi -vcodec mpeg4 -q:v 1 -max_muxing_queue_size 2048 -an full-rigth-codecs.avi

## Container setup and video processing

The following section assumes that OpenFace was installed using docker.

As first step, run the container with the following command:
* `docker run -it --rm --name openface --mount type=bind,source=/home/michele/Desktop/Thesis/media/files,target=/home/openface-build/files algebr/openface:latest`
* Substitute /home/michele/Desktop/Thesis/media/files with the mounting point
* The current working directory must contain a directory **files/**
    * This directory is bound with the --mount option to the files/ directory present in the docker container
    * This directory is shared between the file system of the host and the one of the container

Now we can launch the OpenFace executables (present in the container) from outside the container environment with a command similar to the following:
* `docker exec -it openface build/bin/FeatureExtraction -out_dir files/processed -f files/video.avi`
* From python (this notebook) we can call the same process and read the results
* Create the directory files/processed in advance

In [ ]:
# name given to the container
CONTAINER_NAME = 'openface'

# base directory of the container
CONTAINER_BASE_DIR = '/home/openface-build'
# directory with executalbles in the container
CONTAINER_BIN_DIR = os.path.join(CONTAINER_BASE_DIR, 'build/bin')

FILE_DIR = '../media/files'
OUT_DIR = os.path.join(FILE_DIR, 'processed')

CONTAINER_FILE_DIR = os.path.join(CONTAINER_BASE_DIR, 'files')
CONTAINER_OUT_DIR = os.path.join(CONTAINER_FILE_DIR, 'processed')

CONTAINER_EXECUTABLE = os.path.join(CONTAINER_BIN_DIR, 'FeatureExtraction')

def feature_extraction_offline(video_name):
    
    video_path = os.path.join(FILE_DIR, video_name)
    
    # the file must be in FILE_DIR
    if not os.path.isfile(video_path):
        raise FileNotFoundError(video_path)
    
    container_video_path = os.path.join(CONTAINER_FILE_DIR, video_name)
    
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-f', container_video_path,
        '-out_dir', CONTAINER_OUT_DIR,
        # features extracted
        '-pose', '-gaze', '-aus',
        # output tracked video
        '-tracked'
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command,stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
        
    # poll process and show its output
    while True:
        output = proc.stdout.readline()
        
        if output:
            print(output.strip())
            
        if proc.poll() is not None:
            break
    
    return proc

def feature_extraction_online(pipe='files/pipe'):
    
    command = [
        'docker', 'exec', CONTAINER_NAME, CONTAINER_EXECUTABLE,
        '-device', # use default device
        '-pose', '-gaze', '-aus',
        # '-tracked'
        '-of', pipe
    ]
    
    # capture and combine stdout and stderr into one stream and set as text stream
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)    

    print('Starting real-time analysis...')
    print('Open the pipe from the read side to start the feature stream')
    
    return proc

def read_openface_csv(csv_path):
    return pd.read_csv(csv_path, sep=r',\s*', engine='python')

def ffmpeg_convert(in_file, out_file):
    
    # this was the original command that appeared in the notebook
    # ffmpeg -y -i "files/processed/phone.avi" -c:v libx264 -preset slow -crf 22 -pix_fmt yuv420p -c:a libvo_aacenc -b:a 128k "files/phone-processed.mp4"
    command = [
        'ffmpeg', '-y',
        '-i', in_file,
        '-c:v', 'libx264',
        '-crf', '22',
        '-pix_fmt', 'yuv420p',
        '-c:a', 'libvo_aacenc',
        '-b:a', '128k',
        out_file
    ]
    
    proc = subprocess.run(command, capture_output=True)
    
    return proc

def ffmpeg_merge(video_file, audio_file, out_file):
    
    # ffmpeg -i files/phone-processed.mp4 -i score.wav  -c:v copy phone-processed-son.mp4 -y
                
    command = [
        'ffmpeg', '-y',
        '-i', video_file,
        '-i', audio_file,
        '-map', '0:v',
        '-map', '1:a',
        '-c:v', 'copy',
        out_file
    ]
    
    proc = subprocess.run(command, capture_output=True)
    
    return proc

In [ ]:
video_name = 'full-rigth-codecs.avi'

In [ ]:
%time feature_extraction_offline(video_name)

In [ ]:
ffmpeg_convert("files/processed/phone.avi", "files/phone-processed.mp4")

In [ ]:
df = read_openface_csv(os.path.join(OUT_DIR, "full.csv"))
df.head()

## Sonification

### Setup

Here we are using the current **develop** branch of sc3nb.

In [ ]:
import sc3nb as scn

In [ ]:
# start scsynth
sc = scn.startup()
# connect scsynth to the system playback
!jack_connect "SuperCollider:out_1" "system:playback_1"
!jack_connect "SuperCollider:out_2" "system:playback_2"

If this does not work, open QJackCtl and link the nodes in the graph manually.

In [ ]:
sc.exit()

In [ ]:
# test sound output
sc.server.blip()

Here we want to sonify Action Units (AUs).

OpenFace is able to recognize a **subset** of AUs, specifically: 1, 2, 4, 5, 6, 7, 9, 10, 12, 14, 15, 17, 20, 23, 25, 26, 28, and 45.

AUs can be described in two ways, both supported by OpenFace:
* Presence - if AU is visible in the face
    * 0/1 value
    * e.g. AU01_c, for presence of AU 1
* Intensity - how intense is the AU (minimal to maximal) on a 5 point scale
    * \[0, 5\] values
        * 0: not present
        * 1: present at minimum intensity
        * 5: present at maximum intensity
    * e.g. AU01_r, for intensity of AU 1

NOTE: intensity and presence predictors have been trained separately and on slightly different datasets, this means that **the predictions of both might not always be consistent** (e.g. the presence model could be predicting AU as not being present, but the intensity model could be predicting its value above 1).

AU 28 (lip suck) has only the information regarding to the presence.

In this prototype sonification, we will take into account only two AUs:
* AU 4: Brow Lowerer
* AU 28: Lip Suck
    * Note: for this one OpenFace only provides information about the presence

### Non-Realtime

* We can use NRT Synthesis: https://doc.sccode.org/Guides/Non-Realtime-Synthesis.html
    * scsynth (in NRT mode) will process a score file given as an input (containing OSC commands) and generate an audio file.
    * in our case we will generate the score file from the features extracted by OpenFace (CSV file).
    * Commands that are considered asynchronous in realtime behave as synchronous commands in NRT.
        * simply front-load your Score with all the SynthDefs and Buffers, at time 0.0, and then start the audio processing also at time 0.0
            * you might need a slight offset for the audio processing because sort may not know which entries at time 0.0 must come first.
* sc3nb allows you to do NRT synthesis through its Score class (interface to sclang's Score class).
    * The Score.record_nrt class method provides an easy interface to generates an OSC score file and the relative audio file from a **dict** with **timings as keys** and **lists of OSCMessages as values**.
    * bundler.messages() returns a dictionary in this format
    * Score.record_nrt returns a subprocess.CompletedProcess object with the info relative to the completed scsynth NRT process.

In [ ]:
from sc3nb import Score, SynthDef

In [ ]:
# synthdef = SynthDef(
#     "test",
#     r"""{ |out, freq = 440|
#             OffsetOut.ar(out,
#                 SinOsc.ar(freq, 0, 0.2) * Line.kr(1, 0, 0.5, doneAction: Done.freeSelf)
#             )
#         }""",
# )

synthdef = SynthDef(
    "s2",
    r"""{ | freq=400, amp=0.3, num=4, pan=0, lg=0.1, gate=1 |
            Out.ar(0, Pan2.ar(
                Blip.ar(freq.lag(lg),  num) * EnvGen.kr(Env.asr(0.0, 1.0, 1.0), gate, doneAction: Done.freeSelf),
                    pan.lag(lg),
                    amp.lag(lg)
                )
            )
        }"""
)

The recommended way to proceed it to build a Bundler and use it to generate score file and audio rendering instead of sending it to the server.

* freq=400, amp=0.3, num=4, pan=0, lg=0.1, gate=1
* /s_new:
    * synth definition name
    * synth ID
    * add action (0,1,2, 3 or 4 see below)
    * add target ID
    * N*	
int or string	a control index or name
float or int or string	floating point and integer arguments are interpreted as control value. a symbol argument consisting of the letter 'c' or 'a' (for control or audio) followed by the bus's index.

In [ ]:
with sc.server.bundler(send_on_exit=False) as bundler:
    # setup at the beginning of the score
    synthdef.add()
    
    # instantiate synths
    bundler.add(0.0, "/s_new", ["s2", au4_node_id, 0, 0, "amp", 0])
    
    # iterate over dataframe rows
    for _, row in df.iterrows():
        # print (row["timestamp"], row["AU04_c"], row["AU04_r"], row["AU28_c"])
        
        timestamp = row["timestamp"]
        intensity = row["AU04_r"]
        
        # only "max" should be enough (to clip the top part to 0.3)
        amp = scn.linlin(intensity, 0, 1, 0, 0.3, "minmax")   # TODO: exponential mapping
        # map the intensity of the AU in one octave range
        freq = scn.midicps(scn.linlin(intensity, 0, 5, 69, 81))
        
        # print(amp, freq)
        bundler.add(timestamp, "/n_set", [au4_node_id, "amp", amp, 'freq', freq])

    # assumption: the feature rate is constant
    feature_interval = df.iloc[1]['timestamp']
    # The /c_set [0, 0] will close the audio file
    bundler.add(row["timestamp"] + feature_interval, "/c_set", [0, 0])

Generate the score file and render it.

In [ ]:
Score.record_nrt(bundler.messages(), "/tmp/score.osc", "score.wav", header_format="WAV")

In [ ]:
ffmpeg_merge('files/phone-processed.mp4', 'score.wav', 'phone-processed-son.mp4')

### Real-time

Real-time sonification can be used both for video streams recorded real-time from a device (e.g. webcam) or with video files. In this last case, the advantages with respect to NRT are that the user can start the sonification immediately, without having to wait for the whole video to be processed, and that the user could interactively change the parameters of the sonification on the fly.

#### Online usage

When we track the webcam real-time (-device option), OpenFace (FeatureExtraction executable) opens the stream of the specified device and attempts to open a window with a real-time visualization of the extracted features.

If we are using OpenFace from docker, we have to be careful of granting to the container the access to the webcam device and to the xserver (on Linux).
* The **xhost** program is used to add and delete host names or user names to the list allowed to make connections to the X server
    * `xhost +` grants access to everyone, even if they aren't on the list (i.e., access control is turned off)
    * TODO is it a security problem?
* `docker run -it --rm --name openface --mount type=bind,source=$(pwd)/files,target=/home/openface-build/files --device=/dev/video0:/dev/video0 --net=host -e DISPLAY=$DISPLAY algebr/openface:latest`
    * `--device=/dev/video0:/dev/video0`
    * `--net=host`
    * `-e DISPLAY=$DISPLAY`

If we launch Feature extraction with `-device /dev/video0` (or the right device URL), the executable will start to analyze the stream of the camera and start to generate the usual output files, of which we are particularly interested in the CSV file containing the extracted features. OpenFace appends the features to this file while analyzes the stream; after the stream is closed, OpenFace executes a post-processing on the CSV file.

In this case, as we want the computation to be as light as possible, we will launch the executable with the following options: `-pose -gaze -aus`. This way, OpenFace will not generate anything else than the pose, gaze and AUs features (contained in the CSV file); will not generate e.g. the tracked video.

In this case we would like OpenFace to stream those features directly to our python program, so that somehow our program could process them in real-time. These are some Github issues that are relevant for this topic:
* https://github.com/TadasBaltrusaitis/OpenFace/issues/375
* https://github.com/TadasBaltrusaitis/OpenFace/issues/409
* https://github.com/TadasBaltrusaitis/OpenFace/issues/492
* https://github.com/TadasBaltrusaitis/OpenFace/issues/546
* https://github.com/TadasBaltrusaitis/OpenFace/issues/898

Fundamentally, the possible solutions are 2:
* Modify the C++ code of OpenFace, so that the real-time features are sent to our application using a **messaging library**
    * A project do this using the ZeroMQ library, but it only works for windows
    * Modifying the code of OpenFace may slow down the development a lot and it's probably very complex
* Using named pipes (**FIFOs**): the CSV file would be written to a named pipe (a special file)
    * Should be possible on both Windows and Unix, with some differences
    * This solution is mentioned by the developer in the issue https://github.com/TadasBaltrusaitis/OpenFace/issues/409, but nothing more than this is said; nobody tested it. Will it be performant enough for a real-time application?
    * This should be fast to implement
    
FeatureExtraction has a `-of` option to specify the base name of the output files. The base name will be appended with all the extensions (e.g. .csv) to generate the output files; this is the only way to specify a name for the output CSV file, even if indirectly.

The following is a workaround
* It would be much better if FeatureExtraction supported a `--pipe` option for explicitly supporting specifying a named pipe where to send the extracted features, independently from the other possibly generated files.
* I opened an issue on Github here: https://github.com/TadasBaltrusaitis/OpenFace/issues/995

In [ ]:
!mkfifo files/pipe.csv

In [ ]:
# !rm files/pipe.csv

`docker exec -it openface build/bin/FeatureExtraction -device /dev/video0 -pose -gaze -aus -of files/pipe`
* Launching this command will attempt to write the features in the file pipe.csv

In [ ]:
feature_extraction_online()

In [ ]:
# proc.kill() does not work with docker
# we have to use FeatureExt... why?
# !docker exec -it openface ps -el
!docker exec -it openface pkill FeatureExt

In [ ]:
import csv

FIFO = os.path.join(FILE_DIR, 'pipe.csv')

# send synth definitions to the server here?

# instantiate synths
init()

# open FIFO for reading
with open(FIFO, 'r') as fifo:
    
    # the reader attempts to execute fifo.readline() (which is blocking if there are no lines)
    reader = csv.reader(fifo, skipinitialspace=True)
    
    # the first line written to the pipe is the CSV header
    header = next(reader)
    # transform into dictionary of indexes
    header_dict = {label: idx for idx, label in enumerate(header)}
    
    # the loop ends when the pipe is closed from the writing side
    for row in reader:
        # print(row)
        sonify(row, header_dict)
        
    sc.server.free_all()

#### Offline usage

In this case what we want is to simulate real-time execution with a recorded (and preprocessed) video.

Maybe in this case the **streaming simulation** approach would be the most natural (and maybe the best one). Here we would loop over each line of the dataframe and sleep the required amount of time. Every message would be sent using the updated sonification parameters (we need some sort of concurrent construct to ensure thread-safety).

In [ ]:
t0 = time.time()

# recorder = scn.Recorder(path="sonification.wav")
# recorder.start()

# instantiate synths
sc.server.msg("/s_new", ["s2", au4_node_id, 0, 0, "amp", 0])

# iterate over dataframe rows
for _, row in df.iterrows():
    
    # only "max" should be enough (to clip the top part to 0.3)
    amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")   # TODO: exponential mapping
    # map the intensity of the AU in one octave range
    freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))

    # the bundle will play at t0 + timestamp
    with sc.server.bundler(t0 + row.timestamp) as bundler:
        sc.server.msg("/n_set", [au4_node_id, "amp", amp, "freq", freq], bundle=True)
    
    # sleep for the missing time
    waiting_time = t0 + row.timestamp - time.time()
    
    if waiting_time > 0:
        time.sleep(waiting_time)
    
sc.server.free_all()
# recorder.stop()

In [ ]:
sc.server.free_all();

## Sonification Design

AUs recognized by OpenFace:
* 1: Inner Brow Raiser
* 2: Outer Brow Raiser (unilateral)
* 4: Brow Lowerer
* 5: Upper Lid Raiser
* 6: Cheek Raiser
* 7: Lid Tightener
* 9: Nose Wrinkler (usually goes along with 4 and 10)
* 10: Upper Lip Raiser
* 12: Lip Corner Puller
* 14: Dimpler (fossetta)
* 15: Lip Corner Depressor 
* 17: Chin Raiser
* 20: Lip Stretcher
* 23: Lip Tightener (kiss)
* 25: Lips Part (relax Mentalis, antagonist of AU17)
* 26: Jaw Drop (usually goes along with 25)
* 28: Lip Suck (usually along with 26) - OpenFace provides only information about presence
* 45: Blink

Other AUs are related to eye or head movement. For these OpenFace provides information on gaze and head orientation.

Groups of AUs are interpretable as emotions.

Notes:
* How to sonify AU02? Do we have side info?
* All the expressions except for AU02 are symmetric. Asymmetric expression can maybe generate confusion in OpenFace. It is likely that we will have to set a confidence thrashold to discard some of the data (this is also true for cases where improper lighting yields bad predictions
    * Am I missing something?
    * Read free book here: https://imotions.com/blog/facial-action-coding-system/
* Many of the expressions are antagonists: they can't happen at the same time
    * Antagonists
        * 5 - 7
        * 12 - 14 - 15
        * 17 - 25
        * ...
    * We could think of giving the same instrument to the same group

Blinking seems to be a discrete event.

In [ ]:
# df = pd.read_csv("files/processed/phone.csv", sep=r',\s*', engine='python')

In [ ]:
# df[df.AU45_c == 1].AU45_r

sc3-plugins
* http://doc.sccode.org/Classes/CrossoverDistortion.html
* http://doc.sccode.org/Classes/Crest.html
* http://doc.sccode.org/Classes/SVF.html
* http://doc.sccode.org/Classes/OteyPiano.html
* http://doc.sccode.org/Classes/MdaPiano.html
* http://doc.sccode.org/Classes/SineShaper.html
* http://doc.sccode.org/Classes/TwoTube.html

In [ ]:
# install quarks
%sc Quarks.gui

* sInstruments

### Pentatonic eyes + brows
* 1: Inner Brow Raiser
* 2: Outer Brow Raiser (unilateral)
* 4: Brow Lowerer
* 5: Upper Lid Raiser
* 6: Cheek Raiser
* 7: Lid Tightener

#### With s2 (continuous)

In [ ]:
AU01_NODE_ID = 1001
AU02_NODE_ID = 1002
AU04_NODE_ID = 1003
AU05_NODE_ID = 1004
AU06_NODE_ID = 1005
AU07_NODE_ID = 1006

BASE_TONE = 69
PENTATONIC = [
    scn.midicps(BASE_TONE),
    scn.midicps(BASE_TONE + 3),
    scn.midicps(BASE_TONE + 5),
    scn.midicps(BASE_TONE + 7),
    scn.midicps(BASE_TONE + 10),
    scn.midicps(BASE_TONE + 12)
]

def init():
    sc.server.msg("/s_new", ["s2", AU01_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[5], "num", 1])
    sc.server.msg("/s_new", ["s2", AU02_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[4], "num", 1])
    sc.server.msg("/s_new", ["s2", AU04_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[3], "num", 1])
    sc.server.msg("/s_new", ["s2", AU05_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[2], "num", 1])
    sc.server.msg("/s_new", ["s2", AU06_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[0], "num", 1])
    sc.server.msg("/s_new", ["s2", AU07_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[1], "num", 1])
    
def map_intensity(intensity):
    if intensity < 1.0:
        amp = 0
    else:
        db = scn.linlin(intensity, 1, 5, -20, -5, "minmax")
        amp = scn.dbamp(db)
        
    return amp

def sonify(row, header_dict):
    with sc.server.bundler() as bundler:
        intensity = float(row[header_dict["AU01_r"]])
        amp = map_intensity(intensity)
        sc.server.msg("/n_set", [AU01_NODE_ID, "amp", amp], bundle=True)
        
        intensity = float(row[header_dict["AU02_r"]])
        amp = map_intensity(intensity)
        sc.server.msg("/n_set", [AU02_NODE_ID, "amp", amp], bundle=True)
        
        intensity = float(row[header_dict["AU04_r"]])
        amp = map_intensity(intensity)
        sc.server.msg("/n_set", [AU04_NODE_ID, "amp", amp], bundle=True)
        
        intensity = float(row[header_dict["AU05_r"]])
        amp = map_intensity(intensity)
        sc.server.msg("/n_set", [AU05_NODE_ID, "amp", amp], bundle=True)
        
        intensity = float(row[header_dict["AU06_r"]])
        amp = map_intensity(intensity)
        sc.server.msg("/n_set", [AU06_NODE_ID, "amp", amp], bundle=True)
        
        intensity = float(row[header_dict["AU07_r"]])
        amp = map_intensity(intensity)
        sc.server.msg("/n_set", [AU07_NODE_ID, "amp", amp], bundle=True)

In [ ]:
with sc.server.bundler(send_on_exit=False) as bundler:
    # setup at the beginning of the score
    synthdef.add()
    
    # instantiate synths
    bundler.add(0.0, "/s_new", ["s2", AU01_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[5], "num", 1])
    bundler.add(0.0, "/s_new", ["s2", AU02_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[4], "num", 1])
    bundler.add(0.0, "/s_new", ["s2", AU04_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[3], "num", 1])
    bundler.add(0.0, "/s_new", ["s2", AU05_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[2], "num", 1])
    bundler.add(0.0, "/s_new", ["s2", AU06_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[0], "num", 1])
    bundler.add(0.0, "/s_new", ["s2", AU07_NODE_ID, 0, 0, "amp", 0, "freq", PENTATONIC[1], "num", 1])
    
    # iterate over dataframe rows
    for _, row in df.iterrows():
        # print (row["timestamp"], row["AU04_c"], row["AU04_r"], row["AU28_c"])
        
        timestamp = row["timestamp"]
        
        intensity = row["AU01_r"]
        amp = map_intensity(intensity)
        bundler.add(timestamp, "/n_set", [AU01_NODE_ID, "amp", amp])
        
        intensity = row["AU02_r"]
        amp = map_intensity(intensity)
        bundler.add(timestamp, "/n_set", [AU02_NODE_ID, "amp", amp])
        
        intensity = row["AU04_r"]
        amp = map_intensity(intensity)
        bundler.add(timestamp, "/n_set", [AU04_NODE_ID, "amp", amp])
        
        intensity = row["AU05_r"]
        amp = map_intensity(intensity)
        bundler.add(timestamp, "/n_set", [AU05_NODE_ID, "amp", amp])
        
        intensity = row["AU06_r"]
        amp = map_intensity(intensity)
        bundler.add(timestamp, "/n_set", [AU06_NODE_ID, "amp", amp])
        
        intensity = row["AU07_r"]
        amp = map_intensity(intensity)
        bundler.add(timestamp, "/n_set", [AU07_NODE_ID, "amp", amp])
    
    bundler.add(row["timestamp"], "/c_set", [0, 0])

In [ ]:
Score.record_nrt(bundler.messages(), "/tmp/score.osc", "score.wav", header_format="WAV")

In [ ]:
ffmpeg_merge('files/pentatonic-test.avi', 'score.wav', 'pentatonic-test-son-no-processed.avi')

#### With MdaPiano (event based)

In [ ]:
from sc3nb import Score, SynthDef

synthdef = SynthDef(
    "mdapiano",
    r"""{ |freq=440, gate=1, vel=100, amp=1|
        var piano = MdaPiano.ar(
            freq,
            gate,
            vel,
            decay: 0,
            release: 0,
            hard: 0,
            stereo: 0,
            sustain: 0,
            mul: amp
        );
        DetectSilence.ar(piano, 0.01, doneAction:2);
        Out.ar(0, piano);
    }"""
)

In [ ]:
with sc.server.bundler(send_on_exit=False) as bundler:
    # setup at the beginning of the score
    synthdef.add()
    
    old_range_level_01 = 0
    cur_range_level_01 = 0
    
    old_range_level_02 = 0
    cur_range_level_02 = 0
    
    old_range_level_04 = 0
    cur_range_level_04 = 0
    
    old_range_level_05 = 0
    cur_range_level_05 = 0
    
    old_range_level_06 = 0
    cur_range_level_06 = 0
    
    old_range_level_07 = 0
    cur_range_level_07 = 0
    
    # iterate over dataframe rows
    for _, row in df.iterrows():
        
        timestamp = row["timestamp"]
        
        intensity = row["AU01_r"]
        cur_range_level_01 = int(intensity)
        
        if cur_range_level_01 != old_range_level_01:
            if cur_range_level_01 >= 1:
                vel = scn.linlin(cur_range_level_01, 1, 5, 40, 127)
                bundler.add(timestamp, "/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[5], "vel", vel])
            old_range_level_01 = cur_range_level_01
            
        intensity = row["AU02_r"]
        cur_range_level_02 = int(intensity)
        
        if cur_range_level_02 != old_range_level_02:
            if cur_range_level_02 >= 1:
                vel = scn.linlin(cur_range_level_02, 1, 5, 40, 127)
                bundler.add(timestamp, "/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[4], "vel", vel])
            old_range_level_02 = cur_range_level_02
            
        intensity = row["AU04_r"]
        cur_range_level_04 = int(intensity)
        
        if cur_range_level_04 != old_range_level_04:
            if cur_range_level_04 >= 1:
                vel = scn.linlin(cur_range_level_04, 1, 5, 40, 127)
                bundler.add(timestamp, "/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[3], "vel", vel])
            old_range_level_04 = cur_range_level_04
            
        intensity = row["AU05_r"]
        cur_range_level_05 = int(intensity)
        
        if cur_range_level_05 != old_range_level_05:
            if cur_range_level_05 >= 1:
                vel = scn.linlin(cur_range_level_05, 1, 5, 40, 127)
                bundler.add(timestamp, "/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[2], "vel", vel])
            old_range_level_05 = cur_range_level_05
            
        intensity = row["AU06_r"]
        cur_range_level_06 = int(intensity)
        
        if cur_range_level_06 != old_range_level_06:
            if cur_range_level_06 >= 1:
                vel = scn.linlin(cur_range_level_06, 1, 5, 40, 127)
                bundler.add(timestamp, "/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[0], "vel", vel])
            old_range_level_06 = cur_range_level_06
            
        intensity = row["AU07_r"]
        cur_range_level_07 = int(intensity)
        
        if cur_range_level_07 != old_range_level_07:
            if cur_range_level_07 >= 1:
                vel = scn.linlin(cur_range_level_07, 1, 5, 40, 127)
                bundler.add(timestamp, "/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[1], "vel", vel])
            old_range_level_07 = cur_range_level_07
    
    bundler.add(row["timestamp"], "/c_set", [0, 0])

In [ ]:
Score.record_nrt(bundler.messages(), "/tmp/score.osc", "score.wav", header_format="WAV");

In [ ]:
ffmpeg_merge('files/pentatonic-test.avi', 'score.wav', 'pentatonic-test-piano-son-no-processed.avi')

TODO: adjust amplitude

In [ ]:
old_range_level_01 = 0
cur_range_level_01 = 0

old_range_level_02 = 0
cur_range_level_02 = 0

old_range_level_04 = 0
cur_range_level_04 = 0

old_range_level_05 = 0
cur_range_level_05 = 0

old_range_level_06 = 0
cur_range_level_06 = 0

old_range_level_07 = 0
cur_range_level_07 = 0

def init():
    synthdef.add()

def sonify(row, header_dict):
    
    global old_range_level_01
    global cur_range_level_01

    global old_range_level_02
    global cur_range_level_02

    global old_range_level_04
    global cur_range_level_04

    global old_range_level_05
    global cur_range_level_05

    global old_range_level_06
    global cur_range_level_06

    global old_range_level_07
    global cur_range_level_07

    intensity = float(row[header_dict["AU01_r"]])
    cur_range_level_01 = int(intensity)

    if cur_range_level_01 != old_range_level_01:
        if cur_range_level_01 >= 1:
            vel = scn.linlin(cur_range_level_01, 1, 5, 40, 127)
            sc.server.msg("/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[5], "vel", vel])
        old_range_level_01 = cur_range_level_01

    intensity = float(row[header_dict["AU02_r"]])
    cur_range_level_02 = int(intensity)

    if cur_range_level_02 != old_range_level_02:
        if cur_range_level_02 >= 1:
            vel = scn.linlin(cur_range_level_02, 1, 5, 40, 127)
            sc.server.msg("/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[4], "vel", vel])
        old_range_level_02 = cur_range_level_02

    intensity = float(row[header_dict["AU04_r"]])
    cur_range_level_04 = int(intensity)

    if cur_range_level_04 != old_range_level_04:
        if cur_range_level_04 >= 1:
            vel = scn.linlin(cur_range_level_04, 1, 5, 40, 127)
            sc.server.msg("/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[3], "vel", vel])
        old_range_level_04 = cur_range_level_04

    intensity = float(row[header_dict["AU05_r"]])
    cur_range_level_05 = int(intensity)

    if cur_range_level_05 != old_range_level_05:
        if cur_range_level_05 >= 1:
            vel = scn.linlin(cur_range_level_05, 1, 5, 40, 127)
            sc.server.msg("/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[2], "vel", vel])
        old_range_level_05 = cur_range_level_05

    intensity = float(row[header_dict["AU06_r"]])
    cur_range_level_06 = int(intensity)

    if cur_range_level_06 != old_range_level_06:
        if cur_range_level_06 >= 1:
            vel = scn.linlin(cur_range_level_06, 1, 5, 40, 127)
            sc.server.msg("/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[0], "vel", vel])
        old_range_level_06 = cur_range_level_06

    intensity = float(row[header_dict["AU07_r"]])
    cur_range_level_07 = int(intensity)

    if cur_range_level_07 != old_range_level_07:
        if cur_range_level_07 >= 1:
            vel = scn.linlin(cur_range_level_07, 1, 5, 40, 127)
            sc.server.msg("/s_new", ["mdapiano", -1, 0, 0, "freq", PENTATONIC[1], "vel", vel])
        old_range_level_07 = cur_range_level_07
    
def end():
    pass

### Blink 

In [ ]:
from sc3nb import Score, SynthDef

synthdef = SynthDef(
    "drop",
    r"""{ | freq=600, dp=1200, amp=0.5, dur=0.1, pan=0 |
        var sig, env, fch;
        fch = XLine.kr(freq, freq+dp, dur);
        sig = SinOsc.ar(fch);
        env = EnvGen.kr(Env.perc(0.001, dur, curve: -4), 1.0, doneAction: 2);
        Out.ar(0, Pan2.ar(sig, pan, env*amp))
    }"""
)

In [ ]:
with sc.server.bundler(send_on_exit=False) as bundler:
    # setup at the beginning of the score
    synthdef.add()
    
    blinking = False
    
    # iterate over dataframe rows
    for _, row in df.iterrows():
        
        timestamp = row["timestamp"]
        presence = row["AU45_c"]
        
        if blinking:
            if presence == 0:
                blinking = False
        else:
            if presence == 1:
                blinking = True
                bundler.add(timestamp, "/s_new", ["drop", -1, 0, 0])
    
    bundler.add(row["timestamp"], "/c_set", [0, 0])

In [ ]:
with sc.server.bundler(send_on_exit=False) as bundler:
    # setup at the beginning of the score
    synthdef.add()
    
    blinking = False
    
    # iterate over dataframe rows
    for _, row in df.iterrows():
        
        timestamp = row["timestamp"]
        intensity = row["AU45_r"]
        
        if blinking:
            if intensity < 1:
                blinking = False
        elif intensity >= 1:
            blinking = True
            bundler.add(timestamp, "/s_new", ["drop", -1, 0, 0])
    
    bundler.add(row["timestamp"], "/c_set", [0, 0])

In [ ]:
Score.record_nrt(bundler.messages(), "/tmp/score.osc", "score.wav", header_format="WAV");

In [ ]:
ffmpeg_merge('files/aufnahme3_part_1.mp4', 'score.wav', 'blink-aufnahme3_part_1.avi')

### Multi-percussion sound (event-based)

Similar samples
* snares
<!--     * <audio controls src="samples/132417__sajmund__percussion-clave-like-hit.wav"/> -->
<!--     * <audio controls src="samples/277397__earmark-audio__efev1-percussion-snare.wav"/> -->
* high bells
<!--     * <audio controls src="samples/339808__inspectorj__hand-bells-c-db-single.wav"/> -->
<!--     * <audio controls src="samples/360327__inspectorj__triangle-8-hard-hit-a.wav"/> -->
<!--     * <audio controls src="samples/411574__inspectorj__alto-gong-metal-hit-b-h6-xy.wav"/> -->
<!--     * <audio controls src="samples/387715__jagadamba__gong-percussion.wav"/> -->
<!--     * <audio controls src="samples/414563__pjcohen__agogo-bell-low-velocity11.wav"/> -->
<!--     * <audio controls src="samples/public domain/566514__ginijoyce__turning-objects-into-percussion-78.wav"/> -->
<!--     * <audio controls src="samples/public domain/566386__ginijoyce__turning-objects-into-percussion-17.wav"/> -->
* crash
<!--     * <audio controls src="samples/public domain/209874__veiler__pff-chrash-14.wav"/> -->
* electronic
<!--     * <audio controls src="samples/public domain/487662__phonosupf__electronic-percussion-5.wav"/> -->
<!--     * <audio controls src="samples/207919__altemark__space-snare.wav"/> -->
<!--     * <audio controls src="samples/577014__nezuai__cartoon-percussion-3.wav"/> -->
* drums
<!--     * <audio controls src="samples/public domain/439825__twentytwentymusic__electronic-percussion-2.wav"/> -->
    * <audio controls src="samples/public domain/439828__twentytwentymusic__electronic-percussion-6.wav"/>
<!--     * <audio controls src="samples/public domain/439829__twentytwentymusic__electronic-percussion-5.wav"/> -->
* bass drum
<!--     * <audio controls src="samples/138358__minorr__bass-drum-p.wav"/> -->
<!--     * <audio controls src="samples/234746__sonidotv__legno-10.wav"/> -->

Concepts:
* Sounds that usually plays together should be distinguishable
* Sounds that play more often should be less intrusive
* One area should have sounds that are somehow related

<hr>

* 1: Inner Brow Raiser - <audio controls src="samples/360327__inspectorj__triangle-8-hard-hit-a.wav"/>
* 2: Outer Brow Raiser (unilateral) - <audio controls src="samples/339808__inspectorj__hand-bells-c-db-single.wav"/>
* 4: Brow Lowerer - <audio controls src="samples/411574__inspectorj__alto-gong-metal-hit-b-h6-xy.wav"/>

<hr>

* 5: Upper Lid Raiser - <audio controls src="samples/277397__earmark-audio__efev1-percussion-snare.wav"/>
* 6: Cheek Raiser - <audio controls src="samples/public domain/439829__twentytwentymusic__electronic-percussion-5.wav"/>
* 7: Lid Tightener - <audio controls src="samples/132417__sajmund__percussion-clave-like-hit.wav"/>

<hr>

* 9: Nose Wrinkler (usually goes along with 4 and 10) - <audio controls src="samples/577014__nezuai__cartoon-percussion-3.wav"/>
* 10: Upper Lip Raiser - <audio controls src="samples/public domain/566386__ginijoyce__turning-objects-into-percussion-17.wav"/>

<hr>

* 12: Lip Corner Puller - <audio controls src="samples/public domain/566514__ginijoyce__turning-objects-into-percussion-78.wav"/>
* 14: Dimpler - <audio controls src="samples/public domain/487662__phonosupf__electronic-percussion-5.wav"/>
* 15: Lip Corner Depressor - <audio controls src="samples/387715__jagadamba__gong-percussion.wav"/>
* 20: Lip Stretcher - <audio controls src="samples/414563__pjcohen__agogo-bell-low-velocity11.wav"/>

<hr>

* 23: Lip Tightener - <audio controls src="samples/public domain/209874__veiler__pff-chrash-14.wav"/>

<hr>

* 17: Chin Raiser - <audio controls src="samples/public domain/439825__twentytwentymusic__electronic-percussion-2.wav"/>
* 25: Lips Part (relax Mentalis, antagonist of AU17) - <audio controls src="samples/138358__minorr__bass-drum-p.wav"/>
* 26: Jaw Drop (usually goes along with 25) - <audio controls src="samples/234746__sonidotv__legno-10.wav"/>

<hr>

* 28: Lip Suck (usually along with 26) - <audio controls src="samples/207919__altemark__space-snare.wav"/>
    * OpenFace only provides presence information

<hr>

* 45: Blink (drop)

In [ ]:
AU01_SAMPLE_PATH = "samples/360327__inspectorj__triangle-8-hard-hit-a.wav"
AU02_SAMPLE_PATH = "samples/339808__inspectorj__hand-bells-c-db-single.wav"
AU04_SAMPLE_PATH = "samples/411574__inspectorj__alto-gong-metal-hit-b-h6-xy.wav"
AU05_SAMPLE_PATH = "samples/277397__earmark-audio__efev1-percussion-snare.wav"
AU06_SAMPLE_PATH = "samples/public domain/439829__twentytwentymusic__electronic-percussion-5.wav"
AU07_SAMPLE_PATH = "samples/132417__sajmund__percussion-clave-like-hit.wav"
AU09_SAMPLE_PATH = "samples/577014__nezuai__cartoon-percussion-3.wav"
AU10_SAMPLE_PATH = "samples/public domain/566386__ginijoyce__turning-objects-into-percussion-17.wav"
AU12_SAMPLE_PATH = "samples/public domain/566514__ginijoyce__turning-objects-into-percussion-78.wav"
AU14_SAMPLE_PATH = "samples/public domain/487662__phonosupf__electronic-percussion-5.wav"
AU15_SAMPLE_PATH = "samples/387715__jagadamba__gong-percussion.wav"
AU17_SAMPLE_PATH = "samples/public domain/439825__twentytwentymusic__electronic-percussion-2.wav"
AU20_SAMPLE_PATH = "samples/414563__pjcohen__agogo-bell-low-velocity11.wav"
AU23_SAMPLE_PATH = "samples/public domain/209874__veiler__pff-chrash-14.wav"
AU25_SAMPLE_PATH = "samples/138358__minorr__bass-drum-p.wav"
AU26_SAMPLE_PATH = "samples/234746__sonidotv__legno-10.wav"
AU28_SAMPLE_PATH = "samples/207919__altemark__space-snare.wav"

TODO: DetectSilence does not act smoothly. When the synth is freed, the wave is cut and we hear a "toc" sound. It would be better to use an envelope or a line to control the amplitude.

In [ ]:
from sc3nb import Score, SynthDef

drop_def = SynthDef(
    "drop",
    r"""{ | freq=600, dp=1200, amp=0.5, dur=0.1, pan=0 |
        var sig, env, fch;
        fch = XLine.kr(freq, freq+dp, dur);
        sig = SinOsc.ar(fch);
        env = EnvGen.kr(Env.perc(0.001, dur, curve: -4), 1.0, doneAction: 2);
        Out.ar(0, Pan2.ar(sig, pan, env*amp))
    }"""
)

# define custom playbuf synth that frees the synth when the buffer fades out
playbuf1_def = SynthDef(
    "playbuf1",
    r"""{| out=0, bufnum=0, rate=1, amp=0.2, pan=0 |
        var sig;
        sig = PlayBuf.ar(1, bufnum, rate*BufRateScale.kr(bufnum), doneAction:2);
        DetectSilence.ar(sig, 0.1, doneAction:2);
        Out.ar(0, Pan2.ar(sig, pan, amp));
    }"""
)

# Number of channels that the buffer will be. This must be a fixed integer. The architecture of the SynthDef cannot change after it is compiled.
playbuf2_def = SynthDef(
    "playbuf2",
    r"""{| out=0, bufnum=0, rate=1, amp=0.2, pan=0 |
        var sig;
        sig = PlayBuf.ar(2, bufnum, rate*BufRateScale.kr(bufnum), doneAction:2);
        DetectSilence.ar(sig, 0.1, doneAction:2);
        Out.ar(0, Pan2.ar(sig, pan, amp));
    }"""
)

In [ ]:
def map_range(timestamp, old_range_level, cur_range_level, bundler, buf):

    if cur_range_level != old_range_level:
        if cur_range_level >= 1:
            db = scn.linlin(cur_range_level, 1, 5, -20, 0, "minmax")
            amp = scn.dbamp(db)
            
            if buf.channels == 1:
                synth = "playbuf1"
            else:
                synth = "playbuf2"
            
            bundler.add(timestamp, "/s_new", [synth, -1, 0, 0, "bufnum", buf.bufnum, "amp", amp])

In [ ]:
from sc3nb import Buffer

with sc.server.bundler(send_on_exit=False) as bundler:
    # synth defs
    drop_def.add()
    playbuf1_def.add()
    playbuf2_def.add()
    
    buf01 = Buffer().read(AU01_SAMPLE_PATH)
    buf02 = Buffer().read(AU02_SAMPLE_PATH)
    buf04 = Buffer().read(AU04_SAMPLE_PATH)
    buf05 = Buffer().read(AU05_SAMPLE_PATH)
    buf06 = Buffer().read(AU06_SAMPLE_PATH)
    buf07 = Buffer().read(AU07_SAMPLE_PATH)
    buf09 = Buffer().read(AU09_SAMPLE_PATH)
    buf10 = Buffer().read(AU10_SAMPLE_PATH)
    buf12 = Buffer().read(AU12_SAMPLE_PATH)
    buf14 = Buffer().read(AU14_SAMPLE_PATH)
    buf15 = Buffer().read(AU15_SAMPLE_PATH)
    buf17 = Buffer().read(AU17_SAMPLE_PATH)
    buf20 = Buffer().read(AU20_SAMPLE_PATH)
    buf23 = Buffer().read(AU23_SAMPLE_PATH)
    buf25 = Buffer().read(AU25_SAMPLE_PATH)
    buf26 = Buffer().read(AU26_SAMPLE_PATH)
    buf28 = Buffer().read(AU28_SAMPLE_PATH)
    
    old_range_level_01 = cur_range_level_01 = 0
    old_range_level_02 = cur_range_level_02 = 0
    old_range_level_04 = cur_range_level_04 = 0
    old_range_level_05 = cur_range_level_05 = 0
    old_range_level_06 = cur_range_level_06 = 0
    old_range_level_07 = cur_range_level_07 = 0
    old_range_level_09 = cur_range_level_09 = 0
    old_range_level_10 = cur_range_level_10 = 0
    old_range_level_12 = cur_range_level_12 = 0
    old_range_level_14 = cur_range_level_14 = 0
    old_range_level_15 = cur_range_level_15 = 0
    old_range_level_17 = cur_range_level_17 = 0
    old_range_level_20 = cur_range_level_20 = 0
    old_range_level_23 = cur_range_level_23 = 0
    old_range_level_25 = cur_range_level_25 = 0
    old_range_level_26 = cur_range_level_26 = 0
    
    au28_is_on = False
    blinking = False
    
    # iterate over dataframe rows
    for _, row in df.iterrows():
        
        timestamp = row["timestamp"]
        
        cur_range_level_01 = int(row["AU01_r"])
        map_range(timestamp, old_range_level_01, cur_range_level_01, bundler, buf01)
        old_range_level_01 = cur_range_level_01
        
        cur_range_level_02 = int(row["AU02_r"])
        map_range(timestamp, old_range_level_02, cur_range_level_02, bundler, buf02)
        old_range_level_02 = cur_range_level_02
        
        cur_range_level_04 = int(row["AU04_r"])
        map_range(timestamp, old_range_level_04, cur_range_level_04, bundler, buf04)
        old_range_level_04 = cur_range_level_04
        
        cur_range_level_05 = int(row["AU05_r"])
        map_range(timestamp, old_range_level_05, cur_range_level_05, bundler, buf05)
        old_range_level_05 = cur_range_level_05
        
        cur_range_level_06 = int(row["AU06_r"])
        map_range(timestamp, old_range_level_06, cur_range_level_06, bundler, buf06)
        old_range_level_06 = cur_range_level_06
        
        cur_range_level_07 = int(row["AU07_r"])
        map_range(timestamp, old_range_level_07, cur_range_level_07, bundler, buf07)
        old_range_level_07 = cur_range_level_07
        
        cur_range_level_09 = int(row["AU09_r"])
        map_range(timestamp, old_range_level_09, cur_range_level_09, bundler, buf09)
        old_range_level_09 = cur_range_level_09
        
        cur_range_level_10 = int(row["AU10_r"])
        map_range(timestamp, old_range_level_10, cur_range_level_10, bundler, buf10)
        old_range_level_10 = cur_range_level_10
        
        cur_range_level_12 = int(row["AU12_r"])
        map_range(timestamp, old_range_level_12, cur_range_level_12, bundler, buf12)
        old_range_level_12 = cur_range_level_12
        
        cur_range_level_14 = int(row["AU14_r"])
        map_range(timestamp, old_range_level_14, cur_range_level_14, bundler, buf14)
        old_range_level_14 = cur_range_level_14
        
        cur_range_level_15 = int(row["AU15_r"])
        map_range(timestamp, old_range_level_15, cur_range_level_15, bundler, buf15)
        old_range_level_15 = cur_range_level_15
        
        cur_range_level_17 = int(row["AU17_r"])
        map_range(timestamp, old_range_level_17, cur_range_level_17, bundler, buf17)
        old_range_level_17 = cur_range_level_17        
        
        cur_range_level_20 = int(row["AU20_r"])
        map_range(timestamp, old_range_level_20, cur_range_level_20, bundler, buf20)
        old_range_level_20 = cur_range_level_20
        
        cur_range_level_23 = int(row["AU23_r"])
        map_range(timestamp, old_range_level_23, cur_range_level_23, bundler, buf23)
        old_range_level_23 = cur_range_level_23
        
        cur_range_level_25 = int(row["AU25_r"])
        map_range(timestamp, old_range_level_25, cur_range_level_25, bundler, buf25)
        old_range_level_25 = cur_range_level_25
        
        cur_range_level_26 = int(row["AU26_r"])
        map_range(timestamp, old_range_level_26, cur_range_level_26, bundler, buf26)
        old_range_level_26 = cur_range_level_26
        
#         presence = row["AU45_c"]
        
#         if au28_is_on:
#             if presence == 0:
#                 au28_is_on = False
#         elif presence == 1:
#             au28_is_on = True            
#             bundler.add(timestamp, "/s_new", ["playbuf2", -1, 0, 0, "bufnum", buf28.bufnum, "amp", 0.4])
        
        intensity = row["AU45_r"]
        
        if blinking:
            if intensity < 1:
                blinking = False
        elif intensity >= 1:
            blinking = True
            bundler.add(timestamp, "/s_new", ["drop", -1, 0, 0])

    bundler.add(row["timestamp"], "/c_set", [0, 0])

In [ ]:
Score.record_nrt(bundler.messages(), "/tmp/score.osc", "score.wav", header_format="WAV");

In [ ]:
ffmpeg_merge(os.path.join(OUT_DIR, "full.avi"), 'score.wav', '../media/nrt-renderings/full-processed.avi')

### Harmonic emoitions